In [19]:
import sys
sys.path.append('../..')  # Adjust the path to include the parent directory

import numpy as np
from qiskit import QuantumCircuit, qasm2
from qiskit.circuit.library import QFT
from qiskit.transpiler import PassManager
import canopus
import uuid
import os

In [20]:
def canopus_pass(qc, topology, isa):
    if topology == "chain":
        coupling_map = canopus.utils.gene_chain_coupling_map(qc.num_qubits)
    elif topology == "hhex":
        coupling_map = canopus.utils.gene_hhex_coupling_map(qc.num_qubits)
    elif topology == "square":
        coupling_map = canopus.utils.gene_square_coupling_map(qc.num_qubits)
    else:
        raise ValueError(f"Unsupported topology: {topology}")
    
    backend = canopus.CanopusBackend(coupling_map, isa)
    qc_mapped = PassManager(canopus.CanopusMapping(backend)).run(qc)
    return qc_mapped

def sabre_pass(qc, topology):
    if topology == "chain":
        coupling_map = canopus.utils.gene_chain_coupling_map(qc.num_qubits)
    elif topology == "hhex":
        coupling_map = canopus.utils.gene_hhex_coupling_map(qc.num_qubits)
    elif topology == "square":
        coupling_map = canopus.utils.gene_square_coupling_map(qc.num_qubits)
    else:
        raise ValueError(f"Unsupported topology: {topology}")
    
    backend = canopus.CanopusBackend(coupling_map)
    qc_mapped = PassManager(canopus.SabreMapping(backend)).run(qc)
    return qc_mapped


def toqm_pass(qc, topology):
    TOQM_FLAGS = '-defaults -latency Latency_1_2_6 -expander GreedyTopK 10 -queue TrimSlowNodes 2000 1000 -nodeMod GreedyMapper -retain 1'

    input_fname = f"{uuid.uuid4().hex}.qasm"
    qasm2.dump(qc, input_fname)

    if topology == "chain":
        coupling_file = '../../configs/chain.txt'
        coupling_map = canopus.utils.gene_chain_coupling_map(qc.num_qubits)
    elif topology == "hhex":
        coupling_file = '../../configs/hhex.txt'
        coupling_map = canopus.utils.gene_hhex_coupling_map(qc.num_qubits)
    elif topology == "square":
        coupling_file = '../../configs/square.txt'
        coupling_map = canopus.utils.gene_square_coupling_map(qc.num_qubits)
    else:
        raise ValueError(f"Unsupported topology: {topology}")
    
    # create a tmp file
    output_fname = f"{uuid.uuid4().hex}.qasm"

    os.system(f'../mapper {input_fname} {coupling_file} {TOQM_FLAGS} > {output_fname}')

    # Replace 'swp ' with 'swap ' in the output file
    with open(output_fname, 'r') as f:
        content = f.read()
    content = content.replace('swp ', 'swap ')
    with open(output_fname, 'w') as f:
        f.write(content)

    qc_toqm = QuantumCircuit.from_qasm_str(content)

    # remove the temporary files
    os.remove(input_fname)
    os.remove(output_fname)
    return qc_toqm




In [21]:
n = 4
qc = QFT(n, do_swaps=False).decompose()
qc = canopus.rebase_to_tk2(qc)
qc.draw(fold=10000)

┌──────────────────┐                                                                ┌──────────────────┐                  ┌─────────────────┐   ┌───────────────────┐   ┌────────────────┐               
q_0: ─┤ U(7π/2,0,11.192) ├────────────────────────────────────────────────────────────────┤0                 ├──────────────────┤0                ├───┤ U(7π/8,-π/2,3π/2) ├───┤0               ├───────────────
      └┬────────────────┬┘                  ┌─────────────────┐                           │                  │┌────────────────┐│                 │┌──┴───────────────────┴──┐│  Can(0.25,0,0) │┌─────────────┐
q_1: ──┤ U(7π/2,0,5π/8) ├───────────────────┤0                ├───────────────────────────┤                  ├┤0               ├┤  Can(0.125,0,0) ├┤ U(11π/3,0.61548,6.8987) ├┤1               ├┤ U(3π/2,0,π) ├
       ├───────────────┬┘ ┌────────────────┐│                 │┌─────────────────────────┐│  Can(0.0625,0,0) ││  Can(0.25,0,0) ││                 │└──────┬────────────┬─────┘└────────────────┘└─────────────┘
q_2: ──┤ U(3π/2,0,π/2) ├──┤0               ├┤  Can(0.125,0,0) ├┤ U(10.284,9.0593,4.1825) ├┤                  ├┤1               ├┤1                ├───────┤ U(π/2,0,π) ├───────────────────────────────────────
     ┌─┴───────────────┴─┐│  Can(0.25,0,0) ││                 │└─────────────────────────┘│                  │└┬──────────────┬┘└─────────────────┘       └────────────┘                                       
q_3: ┤ U(4.516,-π/2,π/2) ├┤1               ├┤1                ├───────────────────────────┤1                 ├─┤ U(3π/2,0,4π) ├────────────────────────────────────────────────────────────────────────────────
     └───────────────────┘└────────────────┘└─────────────────┘                           └──────────────────┘ └──────────────┘

In [22]:
qc_canopus = canopus_pass(qc, "chain", "cx")
qc_canopus.draw(fold=10000)

┌──────────────────┐                                                ┌──────────────────┐                                                                                                          
q_0 -> 0 ─┤ U(7π/2,0,11.192) ├────────────────────────────────────────────────┤0                 ├────────────────────────────X─────────────────────────────────────────────────────────────────────────────
          └┬────────────────┬┘                      ┌─────────────────┐       │  Can(0.0625,0,0) │      ┌──────────────┐      │ ┌─────────────────┐┌───────────────────┐                                    
q_1 -> 1 ──┤ U(7π/2,0,5π/8) ├───────────────────────┤0                ├─────X─┤1                 ├──────┤ U(3π/2,0,4π) ├──────X─┤0                ├┤ U(7π/8,-π/2,3π/2) ├─X──────────────────────────────────
         ┌─┴────────────────┴┐┌────────────────┐    │  Can(0.125,0,0) │     │ └┬────────────────┬┘┌─────┴──────────────┴────┐   │  Can(0.125,0,0) │└───┬────────────┬──┘ │ ┌────────────────┐               
q_3 -> 2 ┤ U(4.516,-π/2,π/2) ├┤0               ├────┤1                ├─────X──┤0               ├─┤ U(11π/3,0.61548,6.8987) ├─X─┤1                ├────┤ U(π/2,0,π) ├────X─┤0               ├───────────────
         └─┬───────────────┬─┘│  Can(0.25,0,0) │┌───┴─────────────────┴───┐    │  Can(0.25,0,0) │ └─────────────────────────┘ │ └─────────────────┘    └────────────┘      │  Can(0.25,0,0) │┌─────────────┐
q_2 -> 3 ──┤ U(3π/2,0,π/2) ├──┤1               ├┤ U(10.284,9.0593,4.1825) ├────┤1               ├─────────────────────────────X────────────────────────────────────────────┤1               ├┤ U(3π/2,0,π) ├
           └───────────────┘  └────────────────┘└─────────────────────────┘    └────────────────┘                                                                          └────────────────┘└─────────────┘

In [23]:
qc_sabre = sabre_pass(qc, "chain")
qc_sabre.draw(fold=10000)

┌────────────────┐                       ┌─────────────────┐        ┌────────────────┐ ┌─────────────────────────┐                                              ┌────────────────┐┌─────────────┐
q_1 -> 0 ──┤ U(7π/2,0,5π/8) ├───────────────────────┤0                ├────────┤0               ├─┤ U(11π/3,0.61548,6.8987) ├──────────────────────────────────────────────┤1               ├┤ U(3π/2,0,π) ├
         ┌─┴────────────────┴┐┌────────────────┐    │  Can(0.125,0,0) │        │  Can(0.25,0,0) │ └─────────────────────────┘   ┌─────────────────┐    ┌────────────┐      │  Can(0.25,0,0) │└─────────────┘
q_3 -> 1 ┤ U(4.516,-π/2,π/2) ├┤1               ├────┤1                ├─────X──┤1               ├───────────────────────────────┤1                ├────┤ U(π/2,0,π) ├────X─┤0               ├───────────────
         └─┬───────────────┬─┘│  Can(0.25,0,0) │┌───┴─────────────────┴───┐ │ ┌┴────────────────┴┐      ┌──────────────┐        │  Can(0.125,0,0) │┌───┴────────────┴──┐ │ └────────────────┘               
q_2 -> 2 ──┤ U(3π/2,0,π/2) ├──┤0               ├┤ U(10.284,9.0593,4.1825) ├─X─┤1                 ├──────┤ U(3π/2,0,4π) ├──────X─┤0                ├┤ U(7π/8,-π/2,3π/2) ├─X──────────────────────────────────
          ┌┴───────────────┴─┐└────────────────┘└─────────────────────────┘   │  Can(0.0625,0,0) │      └──────────────┘      │ └─────────────────┘└───────────────────┘                                    
q_0 -> 3 ─┤ U(7π/2,0,11.192) ├────────────────────────────────────────────────┤0                 ├────────────────────────────X─────────────────────────────────────────────────────────────────────────────
          └──────────────────┘                                                └──────────────────┘

In [25]:
qc_toqm = toqm_pass(qc, "chain")
qc_toqm.draw(fold=10000)

sh: line 1: 29546 Segmentation fault: 11  ../mapper 7c9af22e1eb344c19e5fcaf5aa790bf9.qasm ../../configs/chain.txt -defaults -latency Latency_1_2_6 -expander GreedyTopK 10 -queue TrimSlowNodes 2000 1000 -nodeMod GreedyMapper -retain 1 > 6d76bbda9546475bb5b5285c87b917ae.qasm
